In [1]:
from music21 import *
import random

In [2]:
def generate_custom_rhythm(time_signature, bpm, num_beats):
    # Create a stream object with the given time signature
    ts = meter.TimeSignature(time_signature)
    rhythm_stream = stream.Stream()
    rhythm_stream.append(ts)

    # Set the tempo in beats per minute
    rhythm_stream.insert(0, tempo.MetronomeMark(number=bpm))

    # Define possible note durations for the main beat
    durations = [duration.Duration(1), duration.Duration(1/2), duration.Duration(1/4)]

    # Calculate the total duration of the bar
    total_duration =  int(time_signature.split('/')[0]) * (4.0 / int(time_signature.split('/')[1]))  # Adjusting the bar length based on the time signature 
    
    # Generate a random rhythm pattern for the specified number of beats (excluding metronome clicks)
    metronome_clicks = int(time_signature.split('/')[0])
    played_beats = 0
    for i in range(metronome_clicks + num_beats):
        if i < metronome_clicks:
            metronome_sound = note.Note("G6")
            metronome_sound.duration = duration.Duration(60.0 / bpm)   # Adjust the duration to match the BPM
            rhythm_stream.append(metronome_sound)
        else:
            if played_beats < num_beats:
                duration_choice = random.choice(durations)
                rhythm_note = note.Note("C4")
                if played_beats == num_beats - 1:
                    # Adjust the last note's duration to fit the remaining time in the bar
                    remaining_duration = total_duration - rhythm_note.duration.quarterLength
                    rhythm_note.duration = duration.Duration(remaining_duration)
                else:
                    rhythm_note.duration = duration_choice
                rhythm_stream.append(rhythm_note)
                played_beats += 1

    return rhythm_stream

In [3]:
def generate_one_beat(midi_file_addr):
    user_time_signature = "4/4"
    user_bpm = 45
    num_beats = 8

    # Create a part for the rhythm
    rhythm_part = stream.Part()

    generated_rhythm = generate_custom_rhythm(user_time_signature, user_bpm, num_beats)
    rhythm_measure = stream.Measure()
    rhythm_measure.append(generated_rhythm)
    rhythm_part.append(rhythm_measure)

    # Create a MIDI file from the rhythm part
    mf = midi.translate.streamToMidiFile(rhythm_part)
    mf.open(midi_file_addr, "wb")
    mf.write()
    mf.close()

In [4]:
midi_practice_suffixes = [
    f"_night_{i}_practice_{j}"
    for i in range(1, 4)
    for j in range(1, 4)
]
midi_practice_suffixes += [
    f"_day_{i}_practice_1"
    for i in range(1, 4)
]
midi_trial_suffixes = [
    f"_{time_of_day}_{i}_trial{j}"
    for i in range(1, 4)
    for j in range(1, 4)
    for time_of_day in ["morning", "night"]
]

In [5]:
random.seed(42)

for file_suffix in (midi_practice_suffixes + midi_trial_suffixes):
    midi_input_name = f"./midi_sources/recreate_beats-{file_suffix}.midi"
    generate_one_beat(midi_input_name)